In [1]:
# 파이썬 ≥3.5 필수
import sys

assert sys.version_info >= (3, 5)

# 코랩에서 실행되는 노트북인가요?
IS_COLAB = "google.colab" in sys.modules

# 사이킷런 ≥0.20 필수
import sklearn

assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras

assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")
    if IS_COLAB:
        print("런타임 > 런타임 유형 변경 메뉴를 선택하고 하드웨어 가속기로 GPU를 고르세요.")

# 공통 모듈 임포트
import numpy as np
import os
from pathlib import Path

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
tf.random.set_seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

# data_for_lstm.csv 파일 존재 확인
data_path = os.path.join(PROJECT_ROOT_DIR, "RX_Rate.csv")
if not os.path.isfile(data_path):
    print("RX_Rate.csv 파일이 존재하지 않습니다.")


def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


import pandas as pd

In [2]:
df = pd.read_csv('RX_Rate.csv')

In [3]:
df.head()

,Unnamed: 0,Date,회차,등록시간,매매기준율,이전대비,송금보내실 때,송금받으실 때,현찰사실 때,현찰파실 때,USD환산율
0,0,20161230,1,08:37:50,"1,208.50",-,"1,219.90","1,197.10","1,229.64","1,187.36",1.0
1,1,20161230,2,08:40:58,"1,208.50",-,"1,219.90","1,197.10","1,229.64","1,187.36",1.0
2,2,20161230,3,08:43:31,"1,208.50",-,"1,219.90","1,197.10","1,229.64","1,187.36",1.0
3,3,20161230,4,08:52:44,"1,208.50",-,"1,219.90","1,197.10","1,229.64","1,187.36",1.0
4,4,20161230,5,08:58:11,"1,208.00",▼0.50,"1,219.40","1,196.60","1,229.14","1,186.86",1.0


In [4]:
df['매매기준율'] = df['매매기준율'].str.replace(',', '').astype(float)
baseRate = df['매매기준율'].values
baseRate

array([1208.5, 1208.5, 1208.5, ..., 1309.5, 1308. , 1309.5])

In [5]:
# plt.figure(figsize=(200,100))
# plt.plot(baseRate)
# plt.show()

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# # Normalize the data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
baseRate = scaler.fit_transform(baseRate.reshape(-1, 1))

# Split the data into sequences of length 50
sequence_length = 50

sequences = []
for i in range(sequence_length, len(baseRate)):
    sequences.append(baseRate[i - sequence_length:i + 1])

# Convert the list of sequences to a numpy array
sequences = np.array(sequences)

# Split the sequences into input (X) and output (y)
X = sequences[:, :-1]
y = sequences[:, -1]

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data to be suitable for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((139539, 50, 1), (34885, 50, 1), (139539, 1), (34885, 1))

In [7]:
# Assume that X_train is your preprocessed (normalized) data
# We have to reshape it back to 2D before inverse transform
X_train_2D = X_train.reshape(-1, 1)

# Use the "scaler" object to inverse transform the data
X_train_raw = scaler.inverse_transform(X_train_2D)

# If you want to reshape it back to the original shape:
X_train_raw = X_train_raw.reshape(X_train.shape)

X_train_raw[0][:5]  # Show the first 5 sequences in raw form

array([[1226.8],
       [1226.8],
       [1226.8],
       [1227. ],
       [1226.7]])

In [8]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(50, 10))
# for i in range(0, X_train.shape[0]):
#   y = X_train[i].flatten()
#   x = range(i * 50, (i+1)*50)
#   if i == 0:
#     plt.plot(x, y, 'b', label='X_train')
#   else:
#     plt.plot(x, y, 'b')


# for i in range(0,  X_test.shape[0]):
#   y = X_test[i].flatten()
#   x = range(X_train.shape[0]*50+i * 50, X_train.shape[0]*50+(i+1)*50)
#   if i == 0:
#     plt.plot(x, y, 'r', label='X_test')
#   else:
#     plt.plot(x, y, 'r')

# for i in range(0, y_train.shape[0]):
#   y = y_train[i]
#   x = (i+1) * 50
#   if i == 0:
#     plt.plot(x, y, 'mo', label='y_train')
#   else:
#     plt.plot(x, y, 'mo')

# for i in range(0, y_test.shape[0]):
#   y = y_test[i]
#   x = (i+1) * 50 +  y_train.shape[0] * 50
#   if i == 0:
#     plt.plot(x, y, 'co', label='y_test')
#   else:
#     plt.plot(x, y, 'co')

# plt.legend()
# plt.show()

In [9]:
def set_optimizers():
    optimizers = []

    optimizers.append(tf.keras.optimizers.Adam())
    optimizers.append(tf.keras.optimizers.SGD())
    optimizers.append(tf.keras.optimizers.RMSprop())
    optimizers.append(tf.keras.optimizers.Adagrad())
    optimizers.append(tf.keras.optimizers.Adadelta())
    optimizers.append(tf.keras.optimizers.Adamax())
    optimizers.append(tf.keras.optimizers.Nadam())
    optimizers.append(tf.keras.optimizers.Ftrl())
    optimizers.append(tf.keras.optimizers.Adafactor())

    return optimizers

len(set_optimizers())

9

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional

def set_models():
    models = []

    models.append(Sequential([
            GRU(50, input_shape=(sequence_length, 1)),
            Dense(1)
    ], name="GRU-50-1"))

    models.append(Sequential([
            GRU(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
            GRU(50),
            Dense(1)
    ], name="GRU-50-50-1"))

    models.append(Sequential([
        Bidirectional(GRU(50), input_shape=(sequence_length, 1)),
        Dense(1)
    ], name="Bidirectional-GRU-50-1"))

    models.append(Sequential([
            GRU(100, input_shape=(sequence_length, 1)),
            Dense(1)
    ], name="GRU-100-1"))

    models.append(Sequential([
            GRU(100, return_sequences=True, input_shape=(X_train.shape[1], 1)),
            GRU(100),
            Dense(1)
    ], name="GRU-100-100-1"))

    models.append(Sequential([
        Bidirectional(GRU(100), input_shape=(sequence_length, 1)),
        Dense(1)
    ], name="Bidirectional-GRU-100-1"))

    return models

len(set_models())

6

In [11]:
results = []
from tensorflow.keras.callbacks import EarlyStopping

for i in range(len(set_models())):
  for j in range(len(set_optimizers())):
    model = set_models()[i]
    optimizer = set_optimizers()[j]
    try:
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        history = model.fit(X_train, y_train, epochs=5, batch_size=400, validation_data=(X_test, y_test),
                  callbacks=[early_stopping])
        print(model)
        print(model.summary())

        if early_stopping.stopped_epoch > 0:
            results.append(
                    f"Model: {model.name}, Optimizer: {optimizer.get_config()['name']}, last lost: {history.history['loss'][-1]}, Early Stopped at Epoch: {early_stopping.stopped_epoch}")
        else:
            results.append(
                    f"Model: {model.name}, Optimizer: {optimizer.get_config()['name']}, last lost: {history.history['loss'][-1]}")
    except Exception as e:
        print(e)
        results.append(
                f"Model: {model.name}, Optimizer: {optimizer.get_config()['name']}, Mean Squared Error: Error")
    finally:
        print(f"Model: {model.name}, Optimizer: {optimizer.get_config()['name']} Finished")

Epoch 1/5
349/349 [==============================] - 8s 6ms/step - loss: 0.0063 - val_loss: 1.3694e-05
Epoch 2/5
349/349 [==============================] - 2s 5ms/step - loss: 9.4275e-06 - val_loss: 7.4225e-06
Epoch 3/5
349/349 [==============================] - 2s 5ms/step - loss: 7.1192e-06 - val_loss: 6.8594e-06
Epoch 4/5
349/349 [==============================] - 2s 5ms/step - loss: 6.6183e-06 - val_loss: 6.2360e-06
Epoch 5/5
349/349 [==============================] - 2s 5ms/step - loss: 6.2455e-06 - val_loss: 5.9848e-06
Model: "GRU-50-1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_16 (GRU)                (None, 50)                7950      
                                                                 
 dense_12 (Dense)            (None, 1)                 51        
                                                                 
Total params: 8001 (31.25 KB)
Trainable params: 8001 (3

In [13]:
losses = []

for result in results:
    losses.append(result)
    print(result)

Model: GRU-50-1, Optimizer: Adam, last lost: 6.245525128178997e-06
Model: GRU-50-1, Optimizer: SGD, last lost: 8.672521289554425e-06
Model: GRU-50-1, Optimizer: RMSprop, last lost: 0.00012124768545618281
Model: GRU-50-1, Optimizer: Adagrad, last lost: 0.00936937890946865
Model: GRU-50-1, Optimizer: Adadelta, last lost: 0.10427102446556091
Model: GRU-50-1, Optimizer: Adamax, last lost: 6.431740985135548e-06
Model: GRU-50-1, Optimizer: Nadam, last lost: 5.524032530956902e-06
Model: GRU-50-1, Optimizer: Ftrl, last lost: 0.11111944168806076
Model: GRU-50-1, Optimizer: Adafactor, last lost: 5.4775073294877075e-06
Model: GRU-50-50-1, Optimizer: Adam, last lost: 4.707890639110701e-06
Model: GRU-50-50-1, Optimizer: SGD, last lost: 1.6886509911273606e-05
Model: GRU-50-50-1, Optimizer: RMSprop, last lost: 0.00024269278219435364
Model: GRU-50-50-1, Optimizer: Adagrad, last lost: 0.004130566027015448
Model: GRU-50-50-1, Optimizer: Adadelta, last lost: 0.1153414323925972
Model: GRU-50-50-1, Optimiz

In [14]:
results.sort(key=lambda x: float(x.split()[-1]))
results

['Model: GRU-50-50-1, Optimizer: Adam, last lost: 4.707890639110701e-06',
 'Model: GRU-100-1, Optimizer: Adam, last lost: 4.766522579302546e-06',
 'Model: GRU-100-100-1, Optimizer: Adam, last lost: 4.789349077327643e-06',
 'Model: GRU-50-50-1, Optimizer: Nadam, last lost: 5.012138444726588e-06',
 'Model: GRU-100-1, Optimizer: Nadam, last lost: 5.165178663446568e-06',
 'Model: GRU-100-1, Optimizer: SGD, last lost: 5.4459974307974335e-06',
 'Model: GRU-50-1, Optimizer: Adafactor, last lost: 5.4775073294877075e-06',
 'Model: GRU-100-100-1, Optimizer: Adamax, last lost: 5.480169420479797e-06',
 'Model: GRU-50-1, Optimizer: Nadam, last lost: 5.524032530956902e-06',
 'Model: GRU-50-50-1, Optimizer: Adamax, last lost: 5.756230166298337e-06',
 'Model: GRU-50-1, Optimizer: Adam, last lost: 6.245525128178997e-06',
 'Model: GRU-100-1, Optimizer: Adamax, last lost: 6.366941761370981e-06',
 'Model: GRU-50-1, Optimizer: Adamax, last lost: 6.431740985135548e-06',
 'Model: GRU-100-100-1, Optimizer: SG